In [9]:
import cv2
import os

# Path to your video file
downloads_path = os.path.expanduser("~")
video_path = os.path.join(downloads_path, "flight_success_1700steps_20250829_105910.mp4")
print(video_path)

# Updated time stamps in seconds - now including 1:22
timestamps = [33, 66, 100, 133, 166]  # 0:33, 1:06, 1:40, 2:13, 2:46

# Open video
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)

# Extract and save frames
for i, timestamp in enumerate(timestamps):
    # Set video position to timestamp
    cap.set(cv2.CAP_PROP_POS_MSEC, timestamp * 1000)
    
    # Read frame
    ret, frame = cap.read()
    
    if ret:
        # Save frame as PNG
        output_filename = f"frame_{timestamp}s.png"
        output_path = os.path.join(downloads_path, output_filename)
        cv2.imwrite(output_path, frame)
        print(f"Saved frame at {timestamp}s as {output_filename}")
    else:
        print(f"Could not extract frame at {timestamp}s")

cap.release()
print("Frame extraction complete!")

/home/oadam/flight_success_1700steps_20250829_105910.mp4
Saved frame at 33s as frame_33s.png
Saved frame at 66s as frame_66s.png
Saved frame at 100s as frame_100s.png
Saved frame at 133s as frame_133s.png
Saved frame at 166s as frame_166s.png
Frame extraction complete!


In [10]:
import cv2
import numpy as np
import os

def find_content_bounds(image):
    """Find the bounding box of non-white content in the image"""
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Find non-white pixels (threshold at 250 to catch near-white pixels too)
    non_white = gray < 250
    
    # Find bounding box
    rows = np.any(non_white, axis=1)
    cols = np.any(non_white, axis=0)
    
    if not np.any(rows) or not np.any(cols):
        return None
    
    top, bottom = np.where(rows)[0][[0, -1]]
    left, right = np.where(cols)[0][[0, -1]]
    
    return top, bottom, left, right

# Path setup
downloads_path = os.path.expanduser("~")
timestamps = [33, 66, 100, 133, 166]  # 0:33, 1:06, 1:40, 2:13, 2:46

# Load all frames and find common crop bounds
frames = []
all_bounds = []

for timestamp in timestamps:
    frame_path = os.path.join(downloads_path, f"frame_{timestamp}s.png")  # Load PNG files
    frame = cv2.imread(frame_path)
    if frame is not None:
        frames.append(frame)
        bounds = find_content_bounds(frame)
        if bounds:
            all_bounds.append(bounds)
        print(f"Loaded frame at {timestamp}s")
    else:
        print(f"Could not load frame at {timestamp}s")

if len(frames) == 5 and len(all_bounds) == 5:
    # Find the common crop area (union of all content areas with some padding)
    top = min(bound[0] for bound in all_bounds) - 10  # Add some padding
    bottom = max(bound[1] for bound in all_bounds) + 10
    left = min(bound[2] for bound in all_bounds) - 10
    right = max(bound[3] for bound in all_bounds) + 10
    
    # Ensure bounds are within image dimensions
    top = max(0, top)
    left = max(0, left)
    bottom = min(frames[0].shape[0], bottom)
    right = min(frames[0].shape[1], right)
    
    print(f"Cropping all frames to: top={top}, bottom={bottom}, left={left}, right={right}")
    
    # Crop all frames to the same area
    cropped_frames = []
    for frame in frames:
        cropped = frame[top:bottom, left:right]
        cropped_frames.append(cropped)
    
    # Concatenate frames horizontally (directly side by side)
    composite_image = np.hstack(cropped_frames)
    
    # Save composite image as PNG
    composite_path = os.path.join(downloads_path, "video_frames_composite_cropped.png")
    cv2.imwrite(composite_path, composite_image)
    print(f"Cropped composite image saved as: video_frames_composite_cropped.png")
    print(f"Composite dimensions: {composite_image.shape[1]}x{composite_image.shape[0]}")
    print(f"Each frame cropped to: {cropped_frames[0].shape[1]}x{cropped_frames[0].shape[0]}")

else:
    print(f"Error: Expected 5 frames, but loaded {len(frames)} frames")

Loaded frame at 33s
Loaded frame at 66s
Loaded frame at 100s
Loaded frame at 133s
Loaded frame at 166s
Cropping all frames to: top=182, bottom=1833, left=422, right=1969
Cropped composite image saved as: video_frames_composite_cropped.png
Composite dimensions: 7735x1651
Each frame cropped to: 1547x1651


In [34]:
import cv2
import os

# Path to your video file
downloads_path = os.path.expanduser("~")
video_path = os.path.join(downloads_path, "flight_success_1700steps_20250829_105910.mp4")
print(video_path)

# Updated time stamps for 5 frames: 33s to 2:46
timestamps = [33, 66, 100, 133, 166]  # 0:33, 1:06, 1:40, 2:13, 2:46

# Open video
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)

# Extract and save frames
for i, timestamp in enumerate(timestamps):
    # Set video position to timestamp
    cap.set(cv2.CAP_PROP_POS_MSEC, timestamp * 1000)
    
    # Read frame
    ret, frame = cap.read()
    
    if ret:
        # Save frame as PNG
        output_filename = f"frame_{timestamp}s.png"
        output_path = os.path.join(downloads_path, output_filename)
        cv2.imwrite(output_path, frame)
        print(f"Saved frame at {timestamp}s as {output_filename}")
    else:
        print(f"Could not extract frame at {timestamp}s")

cap.release()
print("Frame extraction complete!")



/home/oadam/flight_success_1700steps_20250829_105910.mp4
Saved frame at 33s as frame_33s.png
Saved frame at 66s as frame_66s.png
Saved frame at 100s as frame_100s.png
Saved frame at 133s as frame_133s.png
Saved frame at 166s as frame_166s.png
Frame extraction complete!


In [2]:
import cv2
import numpy as np
import os

def find_content_bounds(image):
    """Find the bounding box of non-white content in the image (removes whitespace)"""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    non_white = gray < 250
    rows = np.any(non_white, axis=1)
    cols = np.any(non_white, axis=0)
    
    if not np.any(rows) or not np.any(cols):
        return None
    
    top, bottom = np.where(rows)[0][[0, -1]]
    left, right = np.where(cols)[0][[0, -1]]
    
    return top, bottom, left, right

def crop_axes_and_title_relaxed(image):
    """Remove axes and title with more conservative cropping"""
    height, width = image.shape[:2]
    
    top_crop = int(height * 0.12)
    bottom_crop = int(height * 0.96)
    left_crop = int(width * 0.06)
    right_crop = int(width * 0.97)
    
    return top_crop, bottom_crop, left_crop, right_crop

# Path setup
downloads_path = os.path.expanduser("~")
timestamps = [33, 66, 100, 133, 166]

# Load all frames
frames = []
all_bounds = []

for timestamp in timestamps:
    frame_path = os.path.join(downloads_path, f"frame_{timestamp}s.png")
    frame = cv2.imread(frame_path)
    if frame is not None:
        frames.append(frame)
        bounds = find_content_bounds(frame)
        if bounds:
            all_bounds.append(bounds)
        print(f"Loaded frame at {timestamp}s")
    else:
        print(f"Could not load frame at {timestamp}s")

if len(frames) == 5 and len(all_bounds) == 5:
    # Step 1: Find common whitespace crop area
    top_ws = max(0, min(bound[0] for bound in all_bounds) - 10)
    left_ws = max(0, min(bound[2] for bound in all_bounds) - 10)
    bottom_ws = min(frames[0].shape[0], max(bound[1] for bound in all_bounds) + 10)
    right_ws = min(frames[0].shape[1], max(bound[3] for bound in all_bounds) + 10)
    
    print(f"Step 1 - Removing whitespace")
    
    # Step 2: Remove whitespace from all frames
    whitespace_cropped_frames = [frame[top_ws:bottom_ws, left_ws:right_ws] for frame in frames]
    
    # Step 3: Remove axes and title
    top_ax, bottom_ax, left_ax, right_ax = crop_axes_and_title_relaxed(whitespace_cropped_frames[0])
    
    print(f"Step 2 - Removing axes/title")
    
    # Step 4: Apply cropping to all frames
    final_cropped_frames = [frame[top_ax:bottom_ax, left_ax:right_ax] for frame in whitespace_cropped_frames]
    
    # Step 5: Add new title text matching matplotlib style
    titled_frames = []
    for i, frame in enumerate(final_cropped_frames):
        # Add white space at the top for title
        title_height = 80
        titled_frame = np.ones((frame.shape[0] + title_height, frame.shape[1], 3), dtype=np.uint8) * 255
        titled_frame[title_height:, :] = frame
        
        # Calculate step number
        timestamp = timestamps[i]
        step_number = timestamp * 10
        title_text = f"UAM Simulation - Step {step_number}.0"
        
        # Font settings to match matplotlib fontsize=16, fontweight='bold'
        font = cv2.FONT_HERSHEY_DUPLEX  # Cleaner font, closer to matplotlib
        font_scale = 2  # Adjusted to match fontsize=16
        font_thickness = 3  # Bold weight
        font_color = (0, 0, 0)  # Black text
        
        # Calculate text size and position for centering
        (text_width, text_height), baseline = cv2.getTextSize(title_text, font, font_scale, font_thickness)
        text_x = (titled_frame.shape[1] - text_width) // 2
        text_y = (title_height + text_height) // 2
        
        # Add text
        cv2.putText(titled_frame, title_text, (text_x, text_y), 
                   font, font_scale, font_color, font_thickness, cv2.LINE_AA)
        
        titled_frames.append(titled_frame)
    
    # Step 6: Concatenate frames horizontally
    composite_image = np.hstack(titled_frames)
    
    # Save composite image
    composite_path = os.path.join(downloads_path, "video_frames_composite_final.png")
    cv2.imwrite(composite_path, composite_image)
    print(f"Final composite image saved as: video_frames_composite_final.png")
    print(f"Composite dimensions: {composite_image.shape[1]}x{composite_image.shape[0]}")

else:
    print(f"Error: Expected 5 frames, but loaded {len(frames)} frames")

Loaded frame at 33s
Loaded frame at 66s
Loaded frame at 100s
Loaded frame at 133s
Loaded frame at 166s
Step 1 - Removing whitespace
Step 2 - Removing axes/title
Final composite image saved as: video_frames_composite_final.png
Composite dimensions: 7040x1466
